<a href="https://colab.research.google.com/github/GayanthaL/detection-of-CKD-using-machine-learning-techniques/blob/main/dep_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
from warnings import filterwarnings
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from fancyimpute import IterativeImputer
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [ ]:
#Ignore warnings for more covinience
import warnings
from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Load the dataset using Pandas
import pandas as pd

# Replace 'path_to_your_file' with the actual path to your CSV file in Google Drive
file_path = '/content/drive/My Drive/Research/Sri Lankan CKD Dataset.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(data.head())

Mounted at /content/drive
   id   age  weight gender      cr  cru    al  gl     na    ca    k     cl  \
0   1  72.0    52.0      M  249.43  NaN  34.9 NaN  142.0  2.43  5.0  113.0   
1   2  76.0    52.0      M  216.05  NaN  41.3 NaN  145.0  2.40  4.7  114.0   
2   3  60.0    80.4      M  155.40  NaN  33.7 NaN  135.0  2.00  4.2  103.0   
3   4  65.0    53.4      F  127.50  NaN  41.8 NaN  138.0  2.80  4.4  100.0   
4   5  59.0    60.4      M  172.71  NaN  39.7 NaN  128.0  2.20  3.6   91.0   

     pr  iron     ua   wbc   rbc    hg    plt  class  
0   NaN   NaN  415.8   NaN   NaN   NaN    NaN      1  
1   NaN  13.0    NaN  9.85  3.44  11.3  356.0      1  
2  11.6  12.0    NaN  9.24  4.60   8.4  329.0      1  
3   NaN   NaN  290.7  9.87  4.32  11.7  364.0      1  
4   NaN   NaN  367.8  8.93  4.82  13.8  214.0      1  


In [ ]:
# dropping id column
data.drop('id', axis = 1, inplace = True)

In [ ]:
# Dictionary mapping abbreviated column names to full names
column_name_mapping = {
    "age": "Age",
    "weight": "Weight",
    "gender": "Gender",
    "cr": "Creatinine",
    "cru": "Creatinine Clearance",
    "al": "Albumin",
    "gl": "Glucose",
    "na": "Sodium",
    "ca": "Calcium",
    "k": "Potassium",
    "cl": "Chloride",
    "wbc": "White Blood Cell Count",
    "rbc": "Red Blood Cell Count",
    "hg": "Hemoglobin",
    "plt": "Platelet Count",
    "class": "Class",  # Assuming "class" is the target variable; leave it as is
    # Add any remaining columns if necessary
}

# Rename columns in the DataFrame
data.rename(columns=column_name_mapping, inplace=True)

# Display the updated DataFrame's columns to confirm renaming
print("Updated column names:")
print(data.columns)

Updated column names:
Index(['Age', 'Weight', 'Gender', 'Creatinine', 'Creatinine Clearance',
       'Albumin', 'Glucose', 'Sodium', 'Calcium', 'Potassium', 'Chloride',
       'pr', 'iron', 'ua', 'White Blood Cell Count', 'Red Blood Cell Count',
       'Hemoglobin', 'Platelet Count', 'Class'],
      dtype='object')


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Identify categorical columns (assuming 'gender' is categorical)
categorical_columns = ['Gender']

# Apply Label Encoding
for column in categorical_columns:
    data[column] = label_encoder.fit_transform(data[column])

# Display the first few rows to verify encoding
print(data.head())

    Age  Weight  Gender  Creatinine  Creatinine Clearance  Albumin  Glucose  \
0  72.0    52.0       1      249.43                   NaN     34.9      NaN   
1  76.0    52.0       1      216.05                   NaN     41.3      NaN   
2  60.0    80.4       1      155.40                   NaN     33.7      NaN   
3  65.0    53.4       0      127.50                   NaN     41.8      NaN   
4  59.0    60.4       1      172.71                   NaN     39.7      NaN   

   Sodium  Calcium  Potassium  Chloride    pr  iron     ua  \
0   142.0     2.43        5.0     113.0   NaN   NaN  415.8   
1   145.0     2.40        4.7     114.0   NaN  13.0    NaN   
2   135.0     2.00        4.2     103.0  11.6  12.0    NaN   
3   138.0     2.80        4.4     100.0   NaN   NaN  290.7   
4   128.0     2.20        3.6      91.0   NaN   NaN  367.8   

   White Blood Cell Count  Red Blood Cell Count  Hemoglobin  Platelet Count  \
0                     NaN                   NaN         NaN             N

In [ ]:
# Calculate missing value percentage for each column
missing_percentage = data.isnull().mean() * 100

# Identify columns to drop based on the 80% missing value threshold
threshold = 80
cols_to_drop = missing_percentage[missing_percentage > threshold].index

# Drop the identified columns
data.drop(columns=cols_to_drop, inplace=True)

# Display the remaining columns and their info
print("Columns after dropping those with more than 80% missing values:")
print(data.info())

Columns after dropping those with more than 80% missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     405 non-null    float64
 1   Weight                  402 non-null    float64
 2   Gender                  406 non-null    int64  
 3   Creatinine              390 non-null    float64
 4   Albumin                 207 non-null    float64
 5   Sodium                  366 non-null    float64
 6   Calcium                 91 non-null     float64
 7   Potassium               367 non-null    float64
 8   Chloride                213 non-null    float64
 9   White Blood Cell Count  366 non-null    float64
 10  Red Blood Cell Count    363 non-null    float64
 11  Hemoglobin              373 non-null    float64
 12  Platelet Count          370 non-null    float64
 13  Class                   406 non

In [ ]:
!pip install fancyimpute

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 9.8 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=2fa214408638a087780cf52533fa32bc8dd0ae3ccbf22f924324b91342f08fbb
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11329 sha256=62f5f44e8e74483700b5825a415aa853fca51ea634b464f4edeca5ea9c613d01
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [ ]:
from sklearn.experimental import enable_iterative_imputer  # Enable experimental iterative imputer
from fancyimpute import IterativeImputer

# MICE Imputation using fancyimpute's IterativeImputer
mice_imputer = IterativeImputer(max_iter=10, random_state=42)
data_iter_imputed = pd.DataFrame(mice_imputer.fit_transform(data), columns=data.columns)
print("\nMICE Imputation:")
print(data_iter_imputed.head())


MICE Imputation:
    Age  Weight  Gender  Creatinine  Albumin  Sodium  Calcium  Potassium  \
0  72.0    52.0     1.0      249.43     34.9   142.0     2.43        5.0   
1  76.0    52.0     1.0      216.05     41.3   145.0     2.40        4.7   
2  60.0    80.4     1.0      155.40     33.7   135.0     2.00        4.2   
3  65.0    53.4     0.0      127.50     41.8   138.0     2.80        4.4   
4  59.0    60.4     1.0      172.71     39.7   128.0     2.20        3.6   

   Chloride  White Blood Cell Count  Red Blood Cell Count  Hemoglobin  \
0     113.0                7.898541              3.990003   11.093988   
1     114.0                9.850000              3.440000   11.300000   
2     103.0                9.240000              4.600000    8.400000   
3     100.0                9.870000              4.320000   11.700000   
4      91.0                8.930000              4.820000   13.800000   

   Platelet Count  Class  
0      262.760974    1.0  
1      356.000000    1.0  
2    

In [ ]:
data_iter_imputed = data_iter_imputed.drop(['Weight','Gender','Platelet Count'], axis = 1)

In [ ]:
data_iter_imputed.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     406 non-null    float64
 1   Creatinine              406 non-null    float64
 2   Albumin                 406 non-null    float64
 3   Sodium                  406 non-null    float64
 4   Calcium                 406 non-null    float64
 5   Potassium               406 non-null    float64
 6   Chloride                406 non-null    float64
 7   White Blood Cell Count  406 non-null    float64
 8   Red Blood Cell Count    406 non-null    float64
 9   Hemoglobin              406 non-null    float64
 10  Class                   406 non-null    float64
dtypes: float64(11)
memory usage: 35.0 KB


In [ ]:
#save to csv data_iter_imputed
data_iter_imputed.to_csv('/content/drive/MyDrive/Data_mining/data_iter_imputed.csv', index=False)

In [ ]:
# Import necessary libraries for scaling
from sklearn.preprocessing import RobustScaler

# Initialize the RobustScaler
scaler = RobustScaler()

# Scale the Iterative Imputer (MICE) imputed data
X_iter_scaled = pd.DataFrame(scaler.fit_transform(data_iter_imputed.drop('Class', axis=1)), columns=data_iter_imputed.columns[:-1])
y_iter = data_iter_imputed['Class']

print("\nIterative Imputer (MICE) Scaled Data:")
print(X_iter_scaled.head())


Iterative Imputer (MICE) Scaled Data:
        Age  Creatinine   Albumin    Sodium   Calcium  Potassium  Chloride  \
0  0.963855    0.484127 -0.553580  0.666667  0.769735   0.623225  1.872811   
1  1.156627    0.345168  0.748350  1.166667  0.630798   0.350498  2.018280   
2  0.385542    0.092687 -0.797692 -0.500000 -1.221685  -0.104047  0.418126   
3  0.626506   -0.023458  0.850063  0.000000  2.483282   0.077771 -0.018279   
4  0.337349    0.164748  0.422867 -1.666667 -0.295443  -0.649502 -1.327496   

   White Blood Cell Count  Red Blood Cell Count  Hemoglobin  
0               -0.067195             -0.411288   -0.414455  
1                0.590416             -0.854839   -0.361290  
2                0.384855              0.080645   -1.109677  
3                0.597155             -0.145161   -0.258065  
4                0.280390              0.258065    0.283871  


In [ ]:
sse = scaler.transform(One_raw)

In [ ]:
sse

array([[ -2.47583103,  -0.55432787,  -7.48024126, -23.        ,
          1.01649313,  -3.85152862, -14.56779045,  -2.52764493,
         -3.74609781,  -3.34401665]])

In [ ]:
X_iter_scaled.describe()

,Age,Creatinine,Albumin,Sodium,Calcium,Potassium,Chloride,White Blood Cell Count,Red Blood Cell Count,Hemoglobin
count,406.000000,406.000000,4.060000e+02,406.000000,406.000000,4.060000e+02,4.060000e+02,4.060000e+02,406.000000,406.000000
mean,-0.045835,0.598042,-1.215273e-01,0.046600,-0.095662,3.752666e-02,1.885882e-02,4.183346e-02,0.034432,-0.002285
std,0.687921,1.276777,1.044563e+00,0.910151,1.027973,6.900851e-01,9.307043e-01,7.817869e-01,0.717356,0.653792
min,-1.927711,-0.426350,-5.415475e+00,-4.000000,-5.621333,-1.831320e+00,-5.982489e+00,-1.667377e+00,-1.604839,-1.729032
25%,-0.481928,-0.204274,-5.387355e-01,-0.500000,-0.550155,-4.676838e-01,-4.546850e-01,-5.342685e-01,-0.467742,-0.464516
50%,0.000000,0.000000,7.227292e-16,0.000000,0.000000,-4.037569e-16,-1.033462e-15,-2.992398e-16,0.000000,0.000000
75%,0.518072,0.795726,4.612645e-01,0.500000,0.449845,5.323162e-01,5.453150e-01,4.657315e-01,0.532258,0.535484
max,1.542169,6.653989,3.433580e+00,2.333333,8.503852,2.805043e+00,5.509524e+00,3.518807e+00,1.685484,1.316129


In [ ]:
# Import necessary libraries for SMOTE
from imblearn.over_sampling import SMOTE

# Function to apply SMOTE
def apply_smote(X, y):
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    return X_resampled, y_resampled

# Apply SMOTE to Iterative Imputer (MICE) scaled data
X_iter_resampled, y_iter_resampled = apply_smote(X_iter_scaled, y_iter)
print("\nIterative Imputer (MICE) Resampled Target Class Distribution:")
print(pd.Series(y_iter_resampled).value_counts())


Iterative Imputer (MICE) Resampled Target Class Distribution:
Class
1.0    223
0.0    223
Name: count, dtype: int64


In [ ]:
X = X_iter_resampled
y = y_iter_resampled

In [ ]:
# Import necessary libraries for splitting the data
from sklearn.model_selection import train_test_split

# Function to split the data
def split_data(X, y, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    return X_train, X_test, y_train, y_test

# Call the split_data function to create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = split_data(X, y) # Assuming X and y are defined as in ipython-input-15-f4f64fd73458


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Best parameters from your GridSearchCV result
best_params = {
    'n_estimators': 100,
    'max_depth': None,
    'min_samples_split': 2
}

# Initialize the Random Forest model with the best parameters
best_model = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    random_state=42
)

# Train the model
best_model.fit(X_train, y_train)

# Evaluate on the training data
y_pred_train = best_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)

# Evaluate on the test data
y_pred_test = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print the results
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("\nClassification Report on Test Data:\n")
print(classification_report(y_test, y_pred_test))


Training Accuracy: 1.0
Test Accuracy: 0.9666666666666667

Classification Report on Test Data:

              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97        50
         1.0       0.97      0.95      0.96        40

    accuracy                           0.97        90
   macro avg       0.97      0.96      0.97        90
weighted avg       0.97      0.97      0.97        90



In [ ]:
X_train.shape

(356, 10)

In [ ]:
X_train.head()

,Age,Creatinine,Albumin,Sodium,Calcium,Potassium,Chloride,White Blood Cell Count,Red Blood Cell Count,Hemoglobin
424,-0.629966,-0.184824,-0.694477,0.500000,0.159929,0.378936,0.734433,-0.191301,0.681141,0.593548
204,0.626506,4.304729,-1.875853,-0.166667,-1.862472,0.350498,-0.163748,-0.467714,-1.129032,-1.058065
396,-0.433735,0.588574,-0.634149,0.500000,0.102963,0.259589,0.709063,0.593785,-0.185484,-0.309677
182,-0.915663,-0.188684,-1.346944,0.333333,-0.188436,-0.194957,1.145469,-1.667377,-0.645161,-0.567742
251,-0.096386,-0.112528,-0.573923,2.000000,0.453616,0.441407,1.979833,-1.145052,0.725806,0.567742


In [ ]:
data_iter_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     406 non-null    float64
 1   Creatinine              406 non-null    float64
 2   Albumin                 406 non-null    float64
 3   Sodium                  406 non-null    float64
 4   Calcium                 406 non-null    float64
 5   Potassium               406 non-null    float64
 6   Chloride                406 non-null    float64
 7   White Blood Cell Count  406 non-null    float64
 8   Red Blood Cell Count    406 non-null    float64
 9   Hemoglobin              406 non-null    float64
 10  Class                   406 non-null    float64
dtypes: float64(11)
memory usage: 35.0 KB


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 356 entries, 424 to 197
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     356 non-null    float64
 1   Creatinine              356 non-null    float64
 2   Albumin                 356 non-null    float64
 3   Sodium                  356 non-null    float64
 4   Calcium                 356 non-null    float64
 5   Potassium               356 non-null    float64
 6   Chloride                356 non-null    float64
 7   White Blood Cell Count  356 non-null    float64
 8   Red Blood Cell Count    356 non-null    float64
 9   Hemoglobin              356 non-null    float64
dtypes: float64(10)
memory usage: 30.6 KB


In [ ]:
y_pred_trainOne = best_model.predict(X_train.iloc[[0]])

In [ ]:
y_pred_trainOne

array([0.])

In [ ]:
y_test.iloc[[0]]

,Class
3,1.0


In [ ]:
import joblib

# Save the trained model to a file
joblib.dump(best_model, "/content/drive/MyDrive/Data_mining/random_forest_model.pkl")

['/content/drive/MyDrive/Data_mining/random_forest_model.pkl']

In [ ]:
loaded_model = joblib.load("/content/drive/MyDrive/Data_mining/random_forest_model.pkl")

In [ ]:
y_pred_trainOne_2= loaded_model.predict(X_test.iloc[[0]])

In [ ]:
y_pred_trainOne_2

array([1.])

--------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
One_raw = data_iter_imputed.iloc[[0]]

In [ ]:
One_raw

,Age,Creatinine,Albumin,Sodium,Calcium,Potassium,Chloride,White Blood Cell Count,Red Blood Cell Count,Hemoglobin,Class
0,72.0,249.43,34.9,142.0,2.43,5.0,113.0,7.898541,3.990003,11.093988,1.0


In [ ]:
One_raw_X_iter_scaled = pd.DataFrame(scaler.transform(One_raw.drop('Class', axis=1)))

In [ ]:
One_raw_X_iter_scaled

,0,1,2,3,4,5,6,7,8,9
0,0.963855,0.484127,-0.55358,0.666667,0.769735,0.623225,1.872811,-0.067195,-0.411288,-0.414455


In [ ]:
y_pred_trainOne_te = loaded_model.predict(One_raw_X_iter_scaled)

In [ ]:
y_pred_trainOne_te

array([1.])

In [ ]:
# Save the scaler
joblib.dump(scaler, "/content/drive/MyDrive/Data_mining/robust_scaler.pkl")

['/content/drive/MyDrive/Data_mining/robust_scaler.pkl']

In [ ]:
import pandas as pd
import os

In [ ]:
# Step 1: Read the CSV file
file_path = "/content/drive/MyDrive/Data_mining/data_iter_imputed.csv"  # Replace with your CSV file's path
data = pd.read_csv(file_path)

In [15]:
data.head(30)

,Age,Creatinine,Albumin,Sodium,Calcium,Potassium,Chloride,White Blood Cell Count,Red Blood Cell Count,Hemoglobin,Class
0,72.0,249.43,34.900000,142.000000,2.430000,5.000000,113.00000,7.898541,3.990003,11.093988,1.0
1,76.0,216.05,41.300000,145.000000,2.400000,4.700000,114.00000,9.850000,3.440000,11.300000,1.0
2,60.0,155.40,33.700000,135.000000,2.000000,4.200000,103.00000,9.240000,4.600000,8.400000,1.0
3,65.0,127.50,41.800000,138.000000,2.800000,4.400000,100.00000,9.870000,4.320000,11.700000,1.0
4,59.0,172.71,39.700000,128.000000,2.200000,3.600000,91.00000,8.930000,4.820000,13.800000,1.0
5,47.0,189.07,35.061912,143.000000,2.316382,5.500000,110.00000,7.740000,2.800000,8.200000,1.0
6,65.0,191.24,36.400000,141.000000,2.100000,3.300000,103.00000,8.400000,3.490000,11.000000,1.0
7,52.0,163.80,33.500000,137.000000,2.500000,3.600000,100.00000,8.210000,4.090000,12.100000,1.0
8,42.0,141.30,41.000000,135.000000,2.200000,5.000000,104.00000,6.120000,4.400000,10.000000,1.0
9,60.0,246.50,35.558928,128.000000,2.216002,4.200000,95.00000,5.290000,3.920000,11.900000,1.0


In [ ]:
datac = data.drop(columns = 'Class', axis = 1)

In [12]:

# Step 4: Create a directory to store individual row files
output_dir = "/content/drive/MyDrive/Data_mining/one raw data"
os.makedirs(output_dir, exist_ok=True)

# Step 5: Iterate over each row and save it as a separate CSV file
for idx, row in datac.iterrows():
    row_df = row.to_frame().T  # Convert the row to a DataFrame
    output_file = os.path.join(output_dir, f"row_{idx + 1}.csv")
    row_df.to_csv(output_file, index=False)
    print(f"Row {idx + 1} saved to {output_file}")

print(f"All rows have been saved as individual CSV files in the '{output_dir}' directory.")

Row 1 saved to /content/drive/MyDrive/Data_mining/one raw data/row_1.csv
Row 2 saved to /content/drive/MyDrive/Data_mining/one raw data/row_2.csv
Row 3 saved to /content/drive/MyDrive/Data_mining/one raw data/row_3.csv
Row 4 saved to /content/drive/MyDrive/Data_mining/one raw data/row_4.csv
Row 5 saved to /content/drive/MyDrive/Data_mining/one raw data/row_5.csv
Row 6 saved to /content/drive/MyDrive/Data_mining/one raw data/row_6.csv
Row 7 saved to /content/drive/MyDrive/Data_mining/one raw data/row_7.csv
Row 8 saved to /content/drive/MyDrive/Data_mining/one raw data/row_8.csv
Row 9 saved to /content/drive/MyDrive/Data_mining/one raw data/row_9.csv
Row 10 saved to /content/drive/MyDrive/Data_mining/one raw data/row_10.csv
Row 11 saved to /content/drive/MyDrive/Data_mining/one raw data/row_11.csv
Row 12 saved to /content/drive/MyDrive/Data_mining/one raw data/row_12.csv
Row 13 saved to /content/drive/MyDrive/Data_mining/one raw data/row_13.csv
Row 14 saved to /content/drive/MyDrive/Data